In [1]:

import numpy as np
import pandas as pd

In [2]:
train_X = pd.read_csv("Train_X.csv")
train_X.head(10)

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Profession_Encoded
0,1303834,23,3,0,0,0,3,13,43
1,7574516,40,10,0,0,0,9,13,11
2,3991815,66,4,1,0,0,4,10,17
3,6256451,41,2,0,0,1,2,12,11
4,5768871,47,11,0,0,0,3,14,18
5,6915937,64,0,0,0,0,0,12,18
6,3954973,58,14,1,0,0,8,12,28
7,1706172,33,2,0,0,0,2,14,16
8,7566849,24,17,0,0,1,11,11,1
9,8964846,23,12,0,0,0,5,13,27


In [3]:
train_Y = pd.read_csv("Train_Y.csv")
train_Y.head(10)

,Risk_Flag
0,0
1,0
2,0
3,1
4,1
5,0
6,0
7,0
8,0
9,0


## Making a node

In [4]:
class Node():
    def __init__(self,feature_index = None, threshold=None , left = None , right = None,info_gain = None , value=None):
    
        # for making decision
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node 
        self.value = value

## Tree class

In [5]:
import numpy as np

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=3, max_depth=3):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, features_to_use, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, features_to_use)
            
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], features_to_use, curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], features_to_use, curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, features_to_use):
        ''' function to find the best split '''
            
        best_split ={}
        max_info_gain = -float("inf")
        best_split["info_gain"] = max_info_gain
        
        for feature_index in features_to_use:
            
            # Extract feature values from the dataset
            feature_values = dataset[:, feature_index]
            # Calculate the number of unique feature values
            num_unique_values = len(np.unique(feature_values))
            # If number of unique feature values is more than 100, choose 100 randomly, else choose all
            if num_unique_values > 100:
                # Randomly select 100 unique feature values from the dataset
                possible_thresholds = np.unique(np.random.choice(feature_values, size=100, replace=False))
            else:
                # Use all unique feature values
                possible_thresholds = np.unique(feature_values)
            
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y, features_to_use):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset, features_to_use)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


In [6]:
import random
def bootstrap_feature(num_of_feature):
    # Generate 5 non-repetitive random numbers from 1 to 9
    random_numbers = random.sample(range(9), 5)
    return random_numbers


In [7]:
def bootstrap_sample(x, y, num_samples):
    # Ensure x and y have the same number of samples
    assert len(x) == len(y), "x and y must have the same number of samples"

    # Get the number of samples in x
    num_samples_total = len(x)

    # Generate random indices for the bootstrap sample
    indices = np.random.choice(num_samples_total, size=num_samples, replace=True)

    # Use the indices to select samples from x and y
    x_new = x[indices]
    y_new = y[indices]

    return x_new, y_new

In [8]:
X_train,Y_train = train_X.values , train_Y.values.reshape(-1,1)

In [14]:
X_train , Y_train = bootstrap_sample(X_train,Y_train,10000)

In [15]:
features_to_use = bootstrap_feature(X_train.shape[0])
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train,features_to_use)

In [16]:
x_test = pd.read_csv('test_x.csv')
y_test = pd.read_csv('test_y.csv')
X_test = x_test.values
Y_test = y_test.values.reshape(-1,1)

In [17]:
X_test , Y_test = bootstrap_sample(X_test,Y_test,10000)

In [18]:
Y_pred = np.array(classifier.predict(X_test))

from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)*100

87.22999999999999